In [1]:
import os

In [2]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass # we don't need to use self keyword
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig( # you are assigning values for the DataIngestionConfig class
            root_dir=config.root_dir,   
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )
        return data_ingestion_config

In [12]:
## component-Data Ingestion
import os
from src.datascience import logger
import shutil

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    

    def copy_data_file(self):
        """
        Copies the file "Temp_dataset.csv" from config.source_URL to config.local_data_file.
        Function returns None
        """
        source_file_path = self.config.source_URL  # Path to the downloaded file
        local_file_path = self.config.local_data_file  # Path to save the copy

        # Ensure the directory for the local file exists
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

        # Copy the file
        shutil.copy(source_file_path, local_file_path)
        logger.info(f"File copied from {source_file_path} to {local_file_path}")

In [13]:
try:
    config =  ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.copy_data_file()
except Exception as e:
    raise e

[2025-03-11 03:23:59,850: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-11 03:23:59,851: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-11 03:23:59,853: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-11 03:23:59,854: INFO: common: created directory at: artifacts]
[2025-03-11 03:23:59,855: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-11 03:23:59,858: INFO: 2943792858: File copied from F:\zizo\MLOPS\datascienceproject\data\Temp_dataset.csv to artifacts/data_ingestion/Temp_dataset.csv]
